In [5]:
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the saved model
model = joblib.load('RFG_Model')

# Load the data
cities = ['2800', '2820', '2830', '2840', '2850', '2900', '2920', '2930', '2942', '2950', '3000', '3460']
dataframes = []

for city in cities:
    filename = f'./data/house_data/house_data_{city}.csv'
    df = pd.read_csv(filename)
    df["City"] = int(city)
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)
data.dropna(inplace=True)

# Select relevant features
features = ['X', 'Y', 'Size', 'Type', 'Energy class', 'City']

# Prepare new data for prediction
new_house_data = pd.DataFrame([[56.034845, 12.591295, 68, 'Ejerlejlighed', 'D', 3000]], columns=features)

# Define a dictionary to store the label encoders
label_encoders = {}

# Apply the necessary preprocessing steps to the new data
for feature in ['Type', 'Energy class', 'City']:
    # Create a new LabelEncoder for each categorical feature
    label_encoders[feature] = LabelEncoder()

    # Fit the label encoder on the training data
    label_encoders[feature].fit(data[feature])

    # Transform the feature values in the new data
    new_house_data[feature] = label_encoders[feature].transform(new_house_data[feature])

# Save the label encoders for future use
joblib.dump(label_encoders, 'label_encoders.pkl')

# Make predictions using the loaded model
predictions = model.predict(new_house_data)

# Print the predictions
print(predictions)


[2225970.]
